In [2]:
import torch.quantization

In [3]:
torch.backends.quantized.engine = 'fbgemm'

In [4]:
torch.backends.quantized.engine

'fbgemm'

## Simple Scaling

In [2]:
import torch

In [24]:
# t = torch.tensor([0.5,0.7,0,-0.7,-0.5])
# t = torch.tensor([-10.3, -5.7, 0.0, 5.7, 10.3])
t = torch.tensor([-10.0, -5.0, 0.0, 5.0, 10.0])
t

tensor([-10.,  -5.,   0.,   5.,  10.])

In [25]:
type(t)

torch.Tensor

In [26]:
tmin, tmax = min(t), max(t)
tmin, tmax

(tensor(-10.), tensor(10.))

In [27]:
qmin, qmax = -128, 127

In [28]:
scale = (tmax - tmin) / (qmax - qmin)
scale

tensor(0.0784)

In [29]:
q = torch.round(t/scale)
q = torch.clamp(q,-128,127)
q = q.to(torch.int8)
q

tensor([-127,  -64,    0,   64,  127], dtype=torch.int8)

In [30]:
dq = q.float() * scale
dq

tensor([-9.9608, -5.0196,  0.0000,  5.0196,  9.9608])

In [31]:
dtype = torch.int8
torch.iinfo(dtype)

iinfo(min=-128, max=127, dtype=int8)

In [32]:
# Absolute Error
torch.abs(dq - t)

tensor([0.0392, 0.0196, 0.0000, 0.0196, 0.0392])

In [45]:
# Percentage error
torch.where(t == 0, 0, 100 * torch.abs(dq - t)/t)

tensor([-0.3922, -0.3922,  0.0000,  0.3922,  0.3922])

## Affine Quantization

In [72]:
# t = torch.rand(10)
t = torch.tensor([0,1,9,10,1000])
t

tensor([   0,    1,    9,   10, 1000])

In [73]:
tmin, tmax = min(t), max(t)
tmin, tmax

(tensor(0), tensor(1000))

In [74]:
qmin, qmax = 0, 255

In [75]:
scale = (tmax - tmin) / (qmax - qmin)
scale

tensor(3.9216)

In [76]:
q = torch.round(t/scale)
q = torch.clamp(q,0,255)
# q = q.to(torch.int8)
q

tensor([  0.,   0.,   2.,   3., 255.])

In [82]:
dq = q.float() * scale
dq

tensor([   0.0000,    0.0000,    7.8431,   11.7647, 1000.0000])

In [69]:
t

tensor([  0,   1,   2,   3, 100])

In [83]:
torch.abs(dq - t)

tensor([0.0000, 1.0000, 1.1569, 1.7647, 0.0000])

In [85]:
# Percentage error
torch.where(t == 0, 0, 100 * torch.abs(dq - t)/t)

tensor([  0.0000, 100.0000,  12.8540,  17.6471,   0.0000])

## Affine Quantization

In [2]:
def quantize(
        x: torch.Tensor,
        scale: float,
        zero_point: int,
        dtype = torch.uint8
    ):
    return round(x/scale + zero_point)
    

In [3]:
def _calculate_scale_and_zeropoint(x,nbits=8):
    minx, maxx = min(x), max(x)

In [10]:
t = torch.tensor([1.345,2.087,3.453])
t

tensor([1.3450, 2.0870, 3.4530])

In [22]:
minx, maxx = min(t), max(t)
minx, maxx

(tensor(1.3450), tensor(3.4530))

In [20]:
scale = (maxx - minx)/255
zero_point = 0 - minx/scale

In [21]:
scale, zero_point

(tensor(0.0083), tensor(-162.7016))